In [ ]:
from rlhfutils.eval_utils import oai_kwargs, load_alldfs, annotate_apfarm, apf_format, load_wgpt, filter_and_sort_df
import pandas as pd
from statistics import mean, stdev
import matplotlib.pyplot as plt
import re
from transformers import AutoTokenizer
from datasets import load_dataset
import openai
from rlhfutils.data import qaform
from transformers import AutoTokenizer
import pandas as pd
from rlhfutils.eval_utils import getapfsft, tok_dist
import matplotlib.pyplot as plt
from rlhfutils.debug_utils import load_rm, progress_rm
import argparse
import nltk
from nltk.tokenize import sent_tokenize
import random
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pickle
from plot_utils import fancyscatter

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
wgs = load_alldfs("../trl-general/genouts/wgpt//", 400, False)
rlcds = load_alldfs("../trl-general/genouts/rlcdfix/", 800, False)
stacks = load_alldfs("../trl-general/genouts/stack/", 400, False)

In [ ]:
# given a dataframe, use rm to get rmscos if it doesn't already have them. 
# then bin / mean at intervals of 10 tokens and return lists for a scatter
def score_rm(indf, rm):
    indf['qstrs'] = [qaform(r['question'], r['response']) for _, r in indf.iterrows()]
    rewards = progress_rm(list(indf['qstrs']), rm, kwargs)
    scos = [a[0]['score'] for a in rewards]
    indf['rewards'] = scos

In [ ]:
rlcds['rlcdsftv3']['question'][0]

In [ ]:
# webgpt get stuff for figures
#adfs = load_alldfs("../trl-general/genouts/wgpt//")
tok, rm, kwargs = load_rm("/home/prasann/Projects/rlhf-exploration/webgpt-llama/models/rewardmodel", 0)

In [ ]:
score_dfs = ['wgptrwscale', 'wgptsft', 'wgptppoorig', 'wgpthkllate', 'wgptthrowlong']
# score_dfs = ['wgptsft', 'wgptppoorig', 'wgpthkllate']
for s in score_dfs:
    score_rm(wgs[s], rm)

In [ ]:
# stack get stuff for figures
tok, rm, kwargs = load_rm("/mnt/data1/prasann/rlhf-exploration/stack-llama/models/rewardsanity/", 0)
#adfs = load_alldfs("../trl-general/genouts/stack/")
score_dfs = ['stacksft', 'stackorigrerun', 'stackhkl']
for s in score_dfs:
    score_rm(stacks[s], rm)

In [ ]:
rlcds.keys()

In [ ]:
# rlcd get stuff for figures
tok, rm, kwargs = load_rm("/home/prasann/Projects/rlhf-exploration/models/rewards/rlcdnormal", 0)

In [ ]:
adfs = load_alldfs("../trl-general/genouts/rlcdfix/")

In [ ]:
score_dfs = ['rlcdorigv3', 'rlcdhkl', 'rlcdlenpen', 'rlcdrwscale']
for s in score_dfs:
    score_rm(rlcds[s], rm)

In [ ]:
rlcdnames = ['rlcdsftv3', 'rlcdhkl', 'rlcdhkl']
stacknames = ['stacksft', 'stackhkl', 'stackhkl']
wgptnames = ['wgptsft', 'wgpthkllate', 'wgpthkllate']

In [ ]:
with open('wgptdata.pkl', 'rb') as f:
    wgs = pickle.load(f)
with open('rlcddata.pkl', 'rb') as f:
    rlcds = pickle.load(f)
with open('stackdata.pkl', 'rb') as f:
    stacks = pickle.load(f)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(13, 3.5), sharey=False)
# fig.suptitle('Initial Pokemon - 1st Generation')
bwidth = 30
sns.color_palette("deep",8)
hexs = sns.color_palette("deep",8).as_hex()
color_1, color2 = hexs[0], hexs[1]
palette ={'sft': color_1, 'ppo': color2}
fancyscatter(wgptnames, 'W-GPT', wgs, axes[0])
fancyscatter(stacknames, 'Stack', stacks, axes[1])
fancyscatter(rlcdnames, 'RLCD', rlcds, axes[2])
fig.tight_layout()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(13, 3.5), sharey=False)
# fig.suptitle('Initial Pokemon - 1st Generation')
bwidth = 30
sns.color_palette("deep",8)
hexs = sns.color_palette("deep",8).as_hex()
color_1, color2 = hexs[0], hexs[1]
palette ={'sft': color_1, 'ppo': color2}
fancyscatter(rlcdlenpen, 'Len Penalty', rlcds, axes[0])
fancyscatter(rlcdhkl, 'HKL', rlcds, axes[1])
fancyscatter(rlcdrwscale, 'RW-SC', rlcds, axes[2])
fig.tight_layout()

In [ ]:
fig.savefig('hkl_lengthbins.pdf')

In [ ]:
import matplotlib as mp

In [ ]:
#plt.savefig("arrows/webgptarrows.pdf", format="pdf", bbox_inches="tight")

In [ ]:
fancyscatter(rlcdnames, 'RLCD', rlcds)
plt.savefig("arrows/rlcdarrows.pdf", format="pdf", bbox_inches="tight")

In [ ]:
fancyscatter(stacknames, 'Stack', stacks)
plt.savefig("arrows/stackarrows.pdf", format="pdf", bbox_inches="tight")